<a href="https://colab.research.google.com/github/imTheDevil/Bridge-cmp-dmg-Detection-Transformer-based/blob/main/%5BPyTorch%5D%20RV_ComponentDetection_Segformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Component Detection Model**

##**References**

**Official Source Code for modeling PyTorch based SegFormer: https://github.com/NVlabs/SegFormer**

**Example notebooks for PyTorch SegFormer: https://colab.research.google.com/drive/1_t3KvF3qg4IJfEhTuftFI1GSlscapNgf?usp=sharing**

**Blog Post on traning SegFormer using custom dataset: https://blog.roboflow.com/how-to-train-segformer-on-a-custom-dataset-with-pytorch-lightning/**

**Cross Entropy Loss Error - https://discuss.pytorch.org/t/cross-entropy-loss-error-on-image-segmentation/60194/12**

##**Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers datasets torch --quiet

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation, SegformerImageProcessor
from datasets import load_metric
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tqdm.notebook import tqdm
from torch.autograd import Variable

In [ ]:
import platform
print("Python version:", platform.python_version())

import json
import sys
import os

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

from PIL import Image
from io import BytesIO

In [ ]:
import logging

# Set up the logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Specify the log file path
log_file = "/content/drive/MyDrive/PyTorch segformer/rerun/component/log_file_wtceloss.log"  # Replace with your desired log file path

# Create a file handler to write log records to the log file in 'append' mode
file_handler = logging.FileHandler(log_file, mode='a')

# Create a formatter for the log records
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

In [ ]:
path_ds = os.path.join('/content/drive/MyDrive','Tokaido Dataset Full') #path to the dataset

In [ ]:
json_filepath = os.path.join(path_ds, "id2label.json")

In [ ]:
#Function to change the path format
def path_correct(path):
  path = '/'.join(path[2:].split('\\'))
  path = os.path.join(path_ds, path)
  return path

In [ ]:
#Loading the training data for component images
#Access the csv file containing the absolute directory paths to each file

col_names = ['image file name', 'component label file name', 'damage label file name', 'depth image file name',
             'camera focal length in mm', 'regular images', 'images containing damage in the RRDR']
ftrain = pd.read_csv(os.path.join(path_ds,'files_train.csv'),names = col_names,delimiter=',')
ftrain.iloc[:,0] = ftrain.iloc[:,0].apply(lambda x: path_correct(x))
ftrain.iloc[:,1] = ftrain.iloc[:,1].apply(lambda x: path_correct(x))
ftrain.iloc[:,2] = ftrain.iloc[:,2].apply(lambda x: path_correct(x))
ftrain.iloc[:,3] = ftrain.iloc[:,3].apply(lambda x: path_correct(x))
train_comp = ftrain.loc[ftrain['regular images']==True, ['image file name', 'component label file name']]
# train_dmg = ftrain.loc[ftrain['images containing damage in the RRDR']==True, ['image file name', 'damage label file name']]

In [ ]:
train_comp.head()

In [ ]:
#Loading the testing data for component images
#Access the csv file containing the absolute directory paths to each file

col_names = ['image file name', 'component label file name', 'damage label file name', 'depth image file name',
             'camera focal length in mm', 'regular images', 'images containing damage in the RRDR']
ftest = pd.read_csv(os.path.join(path_ds,'files_test.csv'),names = col_names,delimiter=',')
ftest.iloc[:,0] = ftest.iloc[:,0].apply(lambda x: path_correct(x))
ftest.iloc[:,1] = ftest.iloc[:,1].apply(lambda x: path_correct(x))
ftest.iloc[:,2] = ftest.iloc[:,2].apply(lambda x: path_correct(x))
ftest.iloc[:,3] = ftest.iloc[:,3].apply(lambda x: path_correct(x))
test_comp = ftest.loc[ftest['regular images']==True, ['image file name', 'component label file name']]
# test_dmg = ftest.loc[ftest['images containing damage in the RRDR']==True, ['image file name', 'damage label file name']]

In [ ]:
test_comp.head()

In [ ]:
train_comp.shape

In [ ]:
test_comp.shape

In [ ]:
np.random.seed(42)
shuffled_train_comp = train_comp.sample(frac=1)
validation_proportion = 0.1
num_validation_samples = int(len(shuffled_train_comp) * validation_proportion)

In [ ]:
# using validation data as 10% of whole training data
train = shuffled_train_comp.iloc[num_validation_samples:]
val = shuffled_train_comp.iloc[:num_validation_samples]
test = test_comp[:1073]

In [ ]:
class SemanticSegmentationDataset(Dataset):

    def __init__(self, root_dir, feature_extractor, dataframe):   #dataframe: dataset of paths of images and corresponding labels

        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.dataframe = dataframe

        data = json.load(open(os.path.join(self.root_dir, 'id2label.json'), "r"))
        self.id2label = {int(k): v for k, v in data.items()}

        self.images = list(self.dataframe['image file name'])
        self.masks = list(self.dataframe['component label file name'])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        image = Image.open(self.images[idx])
        segmentation_map = Image.open(self.masks[idx])

        encoded_inputs = self.feature_extractor(image, segmentation_map, return_tensors="pt") # randomly crop + pad both image and segmentation map to same size

        for k,v in encoded_inputs.items():
          encoded_inputs[k].squeeze_() # removes "batch" dimension

        return encoded_inputs

In [ ]:
feature_extractor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
feature_extractor.do_reduce_labels = False
feature_extractor.size = 216

In [ ]:
train_dataset = SemanticSegmentationDataset(path_ds, feature_extractor, train)
val_dataset = SemanticSegmentationDataset(path_ds, feature_extractor, val)
test_dataset = SemanticSegmentationDataset(path_ds, feature_extractor, test)

In [ ]:
batch_size = 4
num_workers = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

In [ ]:
data = json.load(open(os.path.join(path_ds, 'id2label.json'), "r"))
id2label = {int(k): v for k, v in data.items()}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
#no. of classes
n_classes = len(id2label.keys())
print(n_classes)

In [ ]:
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b0-finetuned-ade-512-512",
    return_dict=False,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, eps=1e-8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model Initialized!")

In [ ]:
SMOOTH = 1e-6

def iou_metric(outputs: torch.Tensor, labels: torch.Tensor):
    # outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    B, H, W = outputs.shape[0], outputs.shape[1], outputs.shape[2]
    mask = (labels!=0) # we don't include the background class (class_id = 0 if any) in the calculation
    outputs = outputs[mask]
    labels = labels[mask]
    outputs = outputs.view(B, H, W)#reshapes outputs and labels above to (B, H, W)
    labels = labels.view(B, H, W)

    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zero if both are 0

    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0

    result = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10

    return result.mean()

In [ ]:
class_weights = [0,
                 1.8603616743670042,
                 7.777137720077815,
                 8.101747530670247,
                 5.774730984703028,
                 34.734403078289915,
                 149.82883160257342,
                 548.8487701219344,
                 283754.9132947977] #These calculation code is shown at the end of notebook

sum_weights = sum(class_weights)
normalized_weights = [w/sum_weights for w in class_weights]
weights_tensor = torch.tensor(class_weights) #converting normalized weights to pytorch tensor

print(normalized_weights)

In [ ]:
import torch
import torch.nn.functional as F

def dice_loss(pred, target, smooth=1.0):
    """
    Computes the Dice loss between predicted and target segmentation masks.

    Args:
        pred: Tensor of shape (batch_size, num_classes, height, width). The predicted segmentation masks.
        target: Tensor of shape (batch_size, height, width). The ground truth segmentation masks.
        smooth: Smoothing parameter to avoid division by zero.

    Returns:
        The Dice loss value as a scalar tensor.
    """
    batch_size, num_classes, height, width = pred.size()

    # Convert integer labels to one-hot encoding
    target = F.one_hot(target, num_classes=num_classes).permute(0, 3, 1, 2).float()

    # Compute softmax probabilities
    pred = torch.softmax(pred, dim=1)

    # Compute intersection and union
    intersection = torch.sum(pred * target, dim=(2, 3))
    union = torch.sum(pred, dim=(2, 3)) + torch.sum(target, dim=(2, 3))

    # Compute Dice coefficient and loss
    dice = (2.0 * intersection + smooth) / (union + smooth)
    loss = 1.0 - dice.mean()

    return loss

In [ ]:
#initializing cross-entropy loss
crit =torch.nn.functional.cross_entropy

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/PyTorch segformer/rerun/component/comprerun_builtin.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
t_loss={}
v_loss={}
t_acc={}
v_acc={}
m_iou={}
pc_iou={}
pixel_acc={}
t_precision = {}
v_precision = {}
t_recall = {}
v_recall = {}
num_classes = 9 #including background class_id = 0

for epoch in range(43, 46):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    accuracies = []
    losses = []
    val_accuracies = []
    val_losses = []
    mean_iou = []
    per_class_iou = [0] * num_classes
    pixel_accuracy = 0
    precision_scores = []
    recall_scores = []
    model.train()
    for idx, batch in enumerate(tqdm(train_dataloader)):

        # get inputs
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(pixel_values=pixel_values, labels=labels)

        # upsample
        upsampled_logits = nn.functional.interpolate(outputs[1], size=labels.shape[-2:], mode="bilinear")
        predicted = upsampled_logits.argmax(dim=1)#for accuracy and iou calculations below

        #accuracy
        mask = (labels != 0) # we don't include the background class (class_id = 0 if any) in the accuracy calculation
        pred_labels = predicted[mask].detach().cpu().numpy() #to numpy
        true_labels = labels[mask].detach().cpu().numpy() #to numpy
        accuracy = accuracy_score(pred_labels, true_labels)

        # loss = dice_loss(upsampled_logits,labels)
        loss = crit(upsampled_logits,labels, weight = weights_tensor.to(device))
        # loss = outputs[0]

        accuracies.append(accuracy)
        losses.append(loss.item())

        loss.backward()

        optimizer.step()

    if(1):
        model.eval()
        with torch.no_grad():
            for idx, batch in enumerate(val_dataloader):

                #get inputs
                pixel_values = batch["pixel_values"].to(device)
                labels = batch["labels"].to(device)

                #forward
                outputs = model(pixel_values=pixel_values, labels=labels)

                #upsample
                upsampled_logits = nn.functional.interpolate(outputs[1], size=labels.shape[-2:], mode="bilinear")
                predicted = upsampled_logits.argmax(dim=1)#for accuracy calculation below

                #accuracy
                mask = (labels != 0) # we don't include the background class (class_id = 0 if any) in the accuracy calculation
                pred_labels = predicted[mask].detach().cpu().numpy()
                true_labels = labels[mask].detach().cpu().numpy()
                accuracy = accuracy_score(pred_labels, true_labels)

                # loss2 = dice_loss(upsampled_logits,labels)
                loss2 = crit(upsampled_logits,labels, weight = weights_tensor.to(device))
                # loss2 = outputs[0]

                val_accuracies.append(accuracy)
                val_losses.append(loss2.item())

                #calculating mean iou
                iou = iou_metric(predicted, labels)
                mean_iou.append(iou.item())

                # Calculating per-class IoU
                for class_id in range(num_classes):
                    true_class = (labels == class_id)
                    pred_class = (predicted == class_id)
                    intersection = torch.logical_and(true_class, pred_class).sum().item()
                    union = torch.logical_or(true_class, pred_class).sum().item()
                    piou = intersection / (union + 1e-6)  #small epsilon to avoid division by zero
                    per_class_iou[class_id] += piou

                # Calculating pixel-wise accuracy
                correct_pixels = (predicted == labels).sum().item()
                total_pixels = labels.nelement()
                pixel_accuracy += correct_pixels / total_pixels

                # Calculating Precision and recall
                precision = precision_score(labels.flatten(), predicted.flatten(), average='weighted', zero_division=1)
                recall = recall_score(labels.flatten(), predicted.flatten(), average='weighted', zero_division=1)

                precision_scores.append(precision)
                recall_scores.append(recall)

    train_loss= np.mean(losses)
    t_loss[epoch]=train_loss
    val_loss = np.mean(val_losses)
    v_loss[epoch]=val_loss
    train_acc = sum(accuracies)/len(accuracies)
    t_acc[epoch]=train_acc
    val_acc = sum(val_accuracies)/len(val_accuracies)
    v_acc[epoch]=val_acc
    iou_score = np.mean(mean_iou)
    m_iou[epoch]=iou_score

    # Calculate average precision and recall for the epoch
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    v_precision[epoch] = avg_precision
    v_recall[epoch] = avg_recall

    # Average per-class IoU and pixel-wise accuracy over all batches
    per_class_iou = [iou / len(val_dataloader) for iou in per_class_iou]
    pc_iou[epoch]=per_class_iou
    pixel_accuracy /= len(val_dataloader)
    pixel_acc[epoch]=pixel_accuracy



    state = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
            'val_loss': val_loss,
            'train_acc':train_acc,
            'val_acc':val_acc,
            'mean_iou':iou_score,
            'mean class iou': per_class_iou,
            'pixel acc': pixel_accuracy,
            'v_precision': avg_precision,
            'v_recall': avg_recall
            }

    torch.save( state, "/content/drive/MyDrive/PyTorch segformer/rerun/component/comprerun_wtce.pt")

    logger.info(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    logger.info(f"Epoch: {epoch}, Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}, Mean IoU: {iou_score:.4f}, per class iou: {per_class_iou}, pixel accuracy: {pixel_accuracy}, val precision: {avg_precision:.4f}, val recall: {avg_recall:.4f}")

    print(f"Train accuracy: {train_acc}\
         Train Loss: {train_loss}\
         Val accuracy: {val_acc}\
         Val Loss: {val_loss}\
         Mean IOU: {iou_score}\
         mean class iou: {per_class_iou}\
         pixel acc: {pixel_accuracy}\
         val precision: {avg_precision}\
         val recall: {avg_recall}")

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize lists to store metrics for the testing phase
num_classes = 9
test_accuracies = []
test_losses = []
test_mean_iou = []
test_per_class_iou = [0] * num_classes
test_pixel_accuracy = 0
test_precision_scores = []
test_recall_scores = []


# Loop over the testing dataset
with torch.no_grad():
    for idx, batch in enumerate(tqdm(test_dataloader)):

        # Get inputs
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(pixel_values=pixel_values, labels=labels)

        # Upsample
        upsampled_logits = nn.functional.interpolate(outputs[1], size=labels.shape[-2:], mode="bilinear")
        predicted = upsampled_logits.argmax(dim=1)

        # Accuracy
        mask = (labels != 0)
        pred_labels = predicted[mask].detach().cpu().numpy()
        true_labels = labels[mask].detach().cpu().numpy()
        accuracy = accuracy_score(pred_labels, true_labels)

        # Loss
        # loss = crit(upsampled_logits, labels, weight=weights_tensor.to(device))
        loss = outputs[0]

        test_accuracies.append(accuracy)
        test_losses.append(loss.item())

        # Calculating mean IoU
        iou = iou_metric(predicted, labels)
        test_mean_iou.append(iou.item())

        # Calculating per-class IoU
        for class_id in range(num_classes):
            true_class = (labels == class_id)
            pred_class = (predicted == class_id)
            intersection = torch.logical_and(true_class, pred_class).sum().item()
            union = torch.logical_or(true_class, pred_class).sum().item()
            piou = intersection / (union + 1e-6)
            test_per_class_iou[class_id] += piou

        # Calculating pixel-wise accuracy
        correct_pixels = (predicted == labels).sum().item()
        total_pixels = labels.nelement()
        test_pixel_accuracy += correct_pixels / total_pixels

        # Calculating Precision and Recall
        precision = precision_score(labels.flatten(), predicted.flatten(), average='weighted', zero_division=1)
        recall = recall_score(labels.flatten(), predicted.flatten(), average='weighted', zero_division=1)
        test_precision_scores.append(precision)
        test_recall_scores.append(recall)

# Calculate average metrics for the testing phase
test_loss = np.mean(test_losses)
test_accuracy = sum(test_accuracies) / len(test_accuracies)
test_mean_iou_score = np.mean(test_mean_iou)

# Calculate average per-class IoU and pixel-wise accuracy over all batches
test_per_class_iou = [iou / len(test_dataloader) for iou in test_per_class_iou]
test_pixel_accuracy /= len(test_dataloader)

# Calculate average precision and recall for the testing phase
test_avg_precision = np.mean(test_precision_scores)
test_avg_recall = np.mean(test_recall_scores)

# Print and log the testing metrics
logger.info(f"Testing Loss: {test_loss:.4f}, Testing Acc: {test_accuracy:.4f}")
logger.info(f"Testing Mean IoU: {test_mean_iou_score:.4f}, Testing per class IoU: {test_per_class_iou}")
logger.info(f"Testing Pixel Accuracy: {test_pixel_accuracy:.4f}, Testing Precision: {test_avg_precision:.4f}, Testing Recall: {test_avg_recall:.4f}")

print(f"Testing accuracy: {test_accuracy}\
     Testing Loss: {test_loss}\
     Testing Mean IOU: {test_mean_iou_score}\
     Testing per class iou: {test_per_class_iou}\
     Testing pixel acc: {test_pixel_accuracy}\
     Testing precision: {test_avg_precision}\
     Testing recall: {test_avg_recall}")


In [ ]:
logger.info(f"Epoch: {31}, Train Loss: {0.4349:.4f}, Train Acc: {0.8464:.4f}")
logger.info(f"Epoch: {31}, Validation Loss: {0.5072:.4f}, Validation Acc: {0.8606:.4f}, Mean IoU: {0.6922:.4f}, per class iou: {[0.0, 0.8765833756333385, 0.49712489417880407, 0.5355120226074326, 0.7473910081556423, 0.3975825755813958, 0.08724355326916003, 0.03485675912864679, 0.0002494959546916023]}, pixel accuracy: {0.8606357957006716}, val precision: {0.8909:.4f}, val recall: {0.8606:.4f}")

In [ ]:
logger.info(f"Epoch: {30}, Train Loss: {0.4618:.4f}, Train Acc: {0.8366:.4f}")
logger.info(f"Epoch: {30}, Validation Loss: {0.7849:.4f}, Validation Acc: {0.7615:.4f}, Mean IoU: {0.5022:.4f}, per class iou: {[0.0, 0.7665766758423768, 0.45672861281235194, 0.38654116769852254, 0.7063124599659741, 0.2157116853566847, 0.075654487920314, 0.02852559773892011, 0.0001952106372549701]}, pixel accuracy: {0.761522436331312}, val precision: {0.8647:.4f}, val recall: {0.7615:.4f}")

In [ ]:
#to get logs from previous sessions
log_file = "/content/drive/MyDrive/PyTorch segformer/rerun/component/log_file_wtceloss.log"  # Replace with the path to your log file

try:
    with open(log_file, 'r') as file:
        log_contents = file.read()
        print(log_contents)
except FileNotFoundError:
    print("Log file not found.")

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/PyTorch segformer/rerun/component/comprerun_builtin.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
iou = checkpoint['mean_iou']
v_loss = checkpoint['val_loss']
t_acc = checkpoint['train_acc']
v_acc = checkpoint['val_acc']

model.eval()

In [ ]:
print('Metrics @ {}epoch-7000train-575val-{}loss are as follows:\n'.format(epoch, 'builtince'))
print('IOU = {}%'.format(iou*100))
print('Train Loss = {}'.format(loss))
print('Val Loss = {}'.format(v_loss))
print('Train Accuracy = {}'.format(t_acc))
print('Val Accuracy = {}'.format(v_acc))

In [ ]:
batchnum = 168
i = 0

for batch in test_dataloader:
    i = i + 1
    images, masks = batch['pixel_values'].to(device), batch['labels'].to(device)
    outputs = model(pixel_values=images, labels=masks)

    logits = outputs[1]

    upsampled_logits = nn.functional.interpolate(
        logits,
        size=masks.shape[-2:],
        mode="bilinear"
    )

    # loss = crit(upsampled_logits, masks, weight=weights_tensor.to(device))
    loss = outputs[0]

    predicted_mask = upsampled_logits.argmax(dim=1).detach().cpu().numpy()
    masks = masks.detach().cpu().numpy()
    original_images = images.detach().cpu().numpy()  # Retrieve original images

    if i == batchnum:
        break


In [ ]:
color_map = {
    0:(0,0,0),
    1:(255,255,255),
    2:(255,0,0),
    3:(0,0,255),
    4:(255,255,0),
    5:(0,0,0),
    6:(255,40,140),
    7:(218,118,27),
    8:(75,75,75)
}

In [ ]:
def prediction_to_vis(prediction):
    vis_shape = prediction.shape + (3,)
    vis = np.zeros(vis_shape)
    for i,c in color_map.items():
        vis[prediction == i] = color_map[i]
    return Image.fromarray(vis.astype(np.uint8))

##**calculating weights**

In [ ]:
#calculating weights of each class

masks = train['component label file name'].to_numpy()

import skimage.io
p_of_class_0_pix=0
p_of_class_1_pix=0
p_of_class_2_pix=0
p_of_class_3_pix=0
p_of_class_4_pix=0
p_of_class_5_pix=0
p_of_class_6_pix=0
p_of_class_7_pix=0
p_of_class_8_pix=0
for i in range(len(masks)):
  img = skimage.io.imread(masks[i])

  # counting the number of pixels
  p_of_class_0_pix = p_of_class_0_pix+((np.sum(img == 0)/(360*640)))
  p_of_class_1_pix = p_of_class_1_pix+((np.sum(img == 1)/(360*640)))
  p_of_class_2_pix = p_of_class_2_pix+((np.sum(img == 2)/(360*640)))
  p_of_class_3_pix = p_of_class_3_pix+((np.sum(img == 3)/(360*640)))
  p_of_class_4_pix = p_of_class_4_pix+((np.sum(img == 4)/(360*640)))
  p_of_class_5_pix = p_of_class_5_pix+((np.sum(img == 5)/(360*640)))
  p_of_class_6_pix = p_of_class_6_pix+((np.sum(img == 6)/(360*640)))
  p_of_class_7_pix = p_of_class_7_pix+((np.sum(img == 7)/(360*640)))
  p_of_class_8_pix = p_of_class_8_pix+((np.sum(img == 8)/(360*640)))

p_of_class_0_pix=p_of_class_0_pix/75.75
p_of_class_1_pix=p_of_class_1_pix/75.75
p_of_class_2_pix=p_of_class_2_pix/75.75
p_of_class_3_pix=p_of_class_3_pix/75.75
p_of_class_4_pix=p_of_class_4_pix/75.75
p_of_class_5_pix=p_of_class_5_pix/75.75
p_of_class_6_pix=p_of_class_6_pix/75.75
p_of_class_7_pix=p_of_class_7_pix/75.75
p_of_class_8_pix=p_of_class_8_pix/75.75

print('percentage of class 0 pixels:', p_of_class_0_pix)
print('percentage of class 1 pixels:', p_of_class_1_pix)
print('percentage of class 2 pixels:', p_of_class_2_pix)
print('percentage of class 3 pixels:', p_of_class_3_pix)
print('percentage of class 4 pixels:', p_of_class_4_pix)
print('percentage of class 5 pixels:', p_of_class_5_pix)
print('percentage of class 6 pixels:', p_of_class_6_pix)
print('percentage of class 7 pixels:', p_of_class_7_pix)
print('percentage of class 8 pixels:', p_of_class_8_pix)

In [ ]:
import numpy as np
import pandas as pd
import skimage.io

# Load your DataFrame containing image and label file paths
# train_file = pd.read_csv('your_train_file.csv')

# List of class labels
class_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]

# Initialize dictionary to store class pixel counts
class_pixel_counts = {label: 0 for label in class_labels}

# Iterate over masks and calculate pixel counts for each class
for mask_path in train['component label file name']:
    img = skimage.io.imread(mask_path)
    for label in class_labels:
        class_pixel_counts[label] += np.sum(img == label)

# Calculate total number of pixels
total_pixels = sum(class_pixel_counts.values())

# Calculate class frequencies and class weights
class_frequencies = {label: count / total_pixels for label, count in class_pixel_counts.items()}
class_weights = {label: 1 / freq for label, freq in class_frequencies.items()}

print('Class Frequencies:', class_frequencies)
print('Class Weights:', class_weights)